In [0]:
import sys, os, time, math
import numpy as np
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

import torch
print(torch.__version__)

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import StepLR

In [0]:
x_train = np.linspace(0, 1, 1000).reshape(-1,1)
y_square = np.square(x_train)
y_root = np.sqrt(x_train)

In [0]:
x_train.shape

In [0]:
batch_size = 1
use_cuda = torch.cuda.is_available()
print(use_cuda)

In [0]:
class NumsDataset(Dataset):
    def __init__(self, data, squares, roots, transform=None):
        self.numbers = data
        self.squares = squares
        self.roots = roots
        self.transform = transform

    def __len__(self):
        return len(self.numbers)

    def __getitem__(self, idx):
        value = torch.tensor(self.numbers[idx], dtype=torch.float)
        square = torch.tensor(self.squares[idx], dtype=torch.float)
        root = torch.tensor(self.roots[idx], dtype=torch.float)
        return (value, square, root)

In [0]:
data = NumsDataset(x_train, y_square, y_root)
data_loader = DataLoader(data, num_workers=0, batch_size=1, shuffle=True)

In [0]:
class Net(nn.Module):
    def __init__(self,
                 n_hidden,
                 n_layers):
        super(Net, self).__init__()
        self.inp = nn.Linear(1, n_hidden)
        self.hidden = nn.Linear(n_hidden, n_hidden)
        self.out = nn.Linear(n_hidden, 1)
        self.n_layers = n_layers

    def forward(self, x):
        x = self.inp(x)
        for i in range(self.n_layers):
            x = self.hidden(x).clamp(min=0)
        x = self.out(x)
        output = torch.sigmoid(x)
        return output

In [0]:
def train(model, device, train_loader, optimizer, epoch, mute=False):
  train_loss = 0.0
  total_train = 0
  correct_train = 0
  model.train()
  for batch_idx, (data, square, _) in enumerate(train_loader):
      data, square = data.to(device), square.to(device)
      optimizer.zero_grad()
      output = model(data)
      loss = F.l1_loss(output, square)
      loss.backward()
      optimizer.step()
      if (batch_idx % 5 == 0 and not mute):
        display('Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()), display_id=str(epoch), update=True)
      
      # accuracy
      _, predicted = torch.max(output.data, 1)
      train_loss += loss.item()
  
  epoch_loss = train_loss / len(train_loader.dataset)
  return epoch_loss

In [0]:
device = torch.device("cuda" if use_cuda else "cpu")

In [0]:
epochs = 10

model = Net(3, 5).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=2, gamma=0.7)

In [0]:
train_losses=[]
for epoch in range(1, epochs + 1):
    epoch_start = time.time()
    train_loss=train(model, device, data_loader, optimizer, epoch)
    train_losses.append(train_loss)
    scheduler.step()

In [0]:
min(train_losses), train_losses.index(min(train_losses)) + 1

In [0]:
print(len(train_losses))
epochs = range(1, len(train_losses) + 1)

plt.plot(epochs, train_losses, 'r', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Аппроксимируем параболу.

- Простая архитектура: полносвязная сеть на 8 скрытых слоёв по 3 нейрона. Наилучший результат на 10 эпохе, с 0.0195 MAE (l1 loss здесь). На графике видно, что максимальное уменьшение ошибки происходит уже на второй эпохе, дальше падение ошибки минимально.
- Увеличиваем количество эпох для той же архитектуры (20). Здесь ошибка уменьшилась незначительно (0.0190): значит, есть смысл менять архитектуру. Посмотрим, какой эффект оказывает изменение количества слоёв.
- На 15 слоях, наблюдаем огромный рост потерь. Не хватает функции активации (relu, as per publication), добавим её.
- Relu снизило ошибку в 10 раз, и теперь она за 20 эпох не достигла минимума. Он достигается примерно за 69 эпох, но все равно не слишком отличается от показателя в 0.019.
- Также стоит отметить случайность: иногда минимальные потери за всё обучение 0.020, 0.019, или снова откатываются на 0.2 без всяких изменений в архитектуре. При этом наилучший показатель достигается в районе 65 эпохи.
- Очевидно, что эксперименты с обычной сетью прямого распространения не слишком помогает. Нужны более радикальные изменения.

# Построим skip-connections сеть (4 слоя)

In [0]:
def approx_relu(inp):
    return 2*(inp.clamp(min=0)) - 4*((inp-0.5).clamp(min=0)) + 2*((inp-1).clamp(min=0))

class Approx_ReLU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, inp):
        return approx_relu(inp)

In [0]:
def approx_output(x, inp):
    sigma_list = torch.cat(tuple(out/coeff for out,coeff in inp.items()), 1)
    sigma = torch.sum(sigma_list)
    return (x-sigma)

class ApproxOut(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, inp):
        return approx_output(x, inp)

In [0]:
arelu = Approx_ReLU()
aout = ApproxOut()

In [0]:
class SkipNet(nn.Module):
    def __init__(self,
                 n_hidden):
        super(SkipNet, self).__init__()
        self.inp = nn.Linear(1, n_hidden)
        self.hidden = nn.Linear(n_hidden, n_hidden)
        self.arelu = arelu
     #   self.out = nn.Linear(4*n_hidden+1, 1)  # in case of sigmoid

    def forward(self, x0):
        x1 = self.arelu(self.inp(x0))
        x2 = self.arelu(self.hidden(x1))
        x3 = self.arelu(self.hidden(x2))
        x4 = self.arelu(self.hidden(x3))
     #   x_out = self.out(torch.cat((x0,x1,x2,x3,x4), 1))
     #   output = torch.sigmoid(x_out)
        to_cat = {x1:2**2, x2:2**4, x3:2**6, x4:2**8}
        output = aout(x0, to_cat)
        return output

In [0]:
device = torch.device("cuda" if use_cuda else "cpu")

In [0]:
epochs = 40

model = SkipNet(4).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=2, gamma=0.7)

In [0]:
train_losses=[]
for epoch in range(1, epochs + 1):
    epoch_start = time.time()
    train_loss=train(model, device, data_loader, optimizer, epoch)
    train_losses.append(train_loss)
    scheduler.step()

In [0]:
min(train_losses), train_losses.index(min(train_losses)) + 1

In [0]:
print(len(train_losses))
epochs = range(1, len(train_losses) + 1)

plt.plot(epochs, train_losses, 'r', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

- В чистом виде результаты не слишком улучшились (0.19). Попробуем восстановить описанную в презентации архитектуру, и для начала реализуем кастомную функцию активации (Approx_ReLU).
- Кастомная функция активации уменьшила ошибку ещё дополнительно (до 0.016). Напишем и кастомную output-функцию (Fm).
- Кастомная функция уменьшила ошибку до 0.015, но при воспроизведении она стала 0.6, потом 0.16, потом 0.08 - случайный элемент всё ещё велик. Тем не менее, для случайной инициализации мы уже превзошли порядок 10^(-1) из презентации, перейдя в зону 10^(-2).
- Далее в tf-версии мы попробуем улучшить точность для обычной сети прямого распространения типичными методами (с помощью tensorboard). Но оценим эту же модель для корня.

In [0]:
def train(model, device, train_loader, optimizer, epoch, mute=False):
  train_loss = 0.0
  total_train = 0
  correct_train = 0
  model.train()
  for batch_idx, (data, _, root) in enumerate(train_loader):
      data, root = data.to(device), root.to(device)
      optimizer.zero_grad()
      output = model(data)
      loss = F.l1_loss(output, root)
      loss.backward()
      optimizer.step()
      if (batch_idx % 5 == 0 and not mute):
        display('Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()), display_id=str(epoch), update=True)
      
      # accuracy
      _, predicted = torch.max(output.data, 1)
      train_loss += loss.item()
  
  epoch_loss = train_loss / len(train_loader.dataset)
  return epoch_loss

In [0]:
epochs = 40

model = SkipNet(4).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=2, gamma=0.7)

In [0]:
train_losses=[]
for epoch in range(1, epochs + 1):
    epoch_start = time.time()
    train_loss=train(model, device, data_loader, optimizer, epoch)
    train_losses.append(train_loss)
    scheduler.step()

In [0]:
min(train_losses), train_losses.index(min(train_losses)) + 1

In [0]:
print(len(train_losses))
epochs = range(1, len(train_losses) + 1)

plt.plot(epochs, train_losses, 'r', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

Just in case:
- https://colab.research.google.com/drive/1-m38ZeNuEAvXkZMmkoiuGOTMVaxP0b4Z - 2.1 (Torch);
- https://colab.research.google.com/drive/1P-Q4D7MdCJTFKAbb71P5iBpYCyQP-QXX - 2.2 (TF).